In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from collections import defaultdict
from shutil import copyfile

In [ ]:
data_dir = "Dataset/Plant_Leaf_Dataset"
batch_size = 20
num_epochs = 10
image_size = 224
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:
full_dataset = datasets.ImageFolder(root=os.path.join(data_dir), transform=transform)

class_names = full_dataset.classes
print("Class Names:", class_names)

Class Names: ['Apple__Apple_scab', 'Apple__Black_rot', 'Apple__Cedar_apple_rust', 'Apple__healthy', 'Background_without_leaves', 'Bean__Blight', 'Bean__Healthy', 'Bean__Mosaic_Virus', 'Bean__Rust', 'Blueberry__healthy', 'Cherry__Powdery_mildew', 'Cherry__healthy', 'Corn__Cercospora_leaf_spot Gray_leaf_spot', 'Corn__Common_rust', 'Corn__Northern_Leaf_Blight', 'Corn__healthy', 'Cowpea__Bacterial_wilt', 'Cowpea__Healthy', 'Cowpea__Mosaic_virus', 'Cowpea__Septoria_leaf_spot', 'Grape__Black_rot', 'Grape__Esca_(Black_Measles)', 'Grape__Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape__healthy', 'Orange__Haunglongbing_(Citrus_greening)', 'Peach__Bacterial_spot', 'Peach__healthy', 'Pepper__bell__Bacterial_spot', 'Pepper__bell__healthy', 'Potato__Early_blight', 'Potato__Late_blight', 'Potato__healthy', 'Raspberry__healthy', 'Soybean__healthy', 'Squash__Powdery_mildew', 'Strawberry__Leaf_scorch', 'Strawberry__healthy', 'Tomato__Bacterial_spot', 'Tomato__Early_blight', 'Tomato__Late_blight', 'Tomato__

In [ ]:
train_size = int(0.7 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size
train_data, val_data, test_data = random_split(full_dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [9]:
#Export Images From Test Dataset For Future Testing

output_dir = "exported_test_images"
os.makedirs(output_dir, exist_ok=True)

class_counts = defaultdict(int)
for image_path, label in test_data.dataset.samples:
    if class_counts[label] < 5:

        class_name = class_names[label]
        class_dir = os.path.join(output_dir, class_name)
        os.makedirs(class_dir, exist_ok=True)
        
        output_path = os.path.join(class_dir, os.path.basename(image_path))
        copyfile(image_path, output_path)
        
        class_counts[label] += 1

    if len(class_counts) == len(class_names) and all(count >= 5 for count in class_counts.values()):
        break

print(f"Exported 5 images per class to {output_dir}")

Exported 5 images per class to exported_test_images


In [12]:
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(full_dataset.classes))
model = model.to(device)

/home/nabil/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nabil/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
def validate_model(model, val_loader):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = 100 * correct / total
    return val_loss, val_accuracy

In [15]:
def train_model():
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = 100 * correct / total

        # Validation phase
        val_loss, val_accuracy = validate_model(model, val_loader)

        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"  Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%")
        print(f"  Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

In [ ]:
def save_model(model, file_path="Models/plant_disease_model.pth"):
    torch.save(model.state_dict(), file_path)
    print(f"Model saved to {file_path}")

In [16]:
if __name__ == "__main__":
    train_model()
    save_model(model, file_path="Models/plant_disease_model.pth")

Epoch [1/10]
  Training Loss: 0.4259, Training Accuracy: 86.86%
  Validation Loss: 0.2228, Validation Accuracy: 92.87%
Epoch [2/10]
  Training Loss: 0.1820, Training Accuracy: 94.08%
  Validation Loss: 0.2957, Validation Accuracy: 91.19%
Epoch [3/10]
  Training Loss: 0.1316, Training Accuracy: 95.76%
  Validation Loss: 0.2336, Validation Accuracy: 94.19%
Epoch [4/10]
  Training Loss: 0.1009, Training Accuracy: 96.82%
  Validation Loss: 0.0952, Validation Accuracy: 97.00%
Epoch [5/10]
  Training Loss: 0.0801, Training Accuracy: 97.45%
  Validation Loss: 0.0722, Validation Accuracy: 97.85%
Epoch [6/10]
  Training Loss: 0.0685, Training Accuracy: 97.85%
  Validation Loss: 0.1055, Validation Accuracy: 97.01%
Epoch [7/10]
  Training Loss: 0.0621, Training Accuracy: 98.01%
  Validation Loss: 0.0645, Validation Accuracy: 98.01%
Epoch [8/10]
  Training Loss: 0.0508, Training Accuracy: 98.34%
  Validation Loss: 0.1090, Validation Accuracy: 97.25%
Epoch [9/10]
  Training Loss: 0.0463, Training A

In [ ]:
def load_model(model, file_path="Models/plant_disease_model.pth"):
    if os.path.exists(file_path):
        model.load_state_dict(torch.load(file_path, map_location=device))
        model = model.to(device)
        print(f"Model loaded from {file_path}")
    else:
        print(f"No model found at {file_path}.")
    return model

loaded_model = models.resnet18(pretrained=False)
num_features = loaded_model.fc.in_features
loaded_model.fc = nn.Linear(num_features, len(full_dataset.classes))
loaded_model = load_model(loaded_model, file_path="Models/plant_disease_model.pth")

Model loaded from Models/plant_disease_model.pth


/home/nabil/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_7556/290024227.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We

In [18]:
def test_model(model, test_loader):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_loss /= len(test_loader)
    test_accuracy = 100 * correct / total
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")
    return test_accuracy

In [19]:
test_accuracy = test_model(loaded_model, test_loader)

Test Loss: 0.0966, Test Accuracy: 97.03%


In [ ]:
from PIL import Image

def predict_image(image_path, model, class_names):

    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)

    image = image.to(device)

    model.eval()

    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        predicted_class = class_names[predicted.item()]

    print(f"Predicted Class: {predicted_class}")
    return predicted_class


In [21]:
class_names = full_dataset.classes

image_path = "exported_test_images/Blueberry__healthy/image (1).jpg"
predict_image(image_path, loaded_model, class_names)


Predicted Class: Blueberry__healthy


'Blueberry__healthy'